In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys, os, django

sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *

django.db.connection.close()

In [2]:
# Get a list of predictions for the unseen documents
y_pred = np.load('data/predictions.npy')

# Load the documents and add the predictions
seen_df = pd.read_csv('data/0a_labelled_documents.csv')
unseen_df = pd.read_csv('data/0b_unlabelled_documents.csv')

df = (pd.concat([seen_df,unseen_df])
      .sort_values('id')
      .sample(frac=1, random_state=1)
      .reset_index(drop=True)
     )

seen_index = df[df['seen']==1].index
unseen_index = df[df['seen']==0].index

df['prediction'] = df['relevant']
df.loc[unseen_index,"prediction"] = y_pred[:,1]
print(df.shape)

# Get just likely relevant documents
df = df[df['prediction']>0.33]
print(df.shape)
df.head()

(699525, 15)
(34160, 15)


,id,content,title,PY,wosarticle__de,wosarticle__wc,relevant,seen,1 - Adaptation,1 - Impact,1 - Mitigation,wosarticle__dt,tslug,UT__UT,prediction
52,4963912,Objective To explore the association between n...,Exploration of the association between meteoro...,2021.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,explorationoftheassociationbetweenmeteorologic...,2-s2.0-85123003389,0.790797
69,4729030,Background. Acute gastroenteritis is a cause o...,ACUTE GASTROENTERITIS IN 3 COMMUNITY-BASED NUR...,1995.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,acutegastroenteritisin3communitybasednursinghomes,WOS:A1995RY60500014,0.714243
77,4695222,To study the risk factors for Plasmodium vivax...,Risk factors for Plasmodium vivax gametocyte c...,2004.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,riskfactorsforplasmodiumvivaxgametocytecarriag...,WOS:000226125300001,0.465032
80,4762134,Background: Despite declining age-adjusted str...,Mortality and Predictors of Death 1 Month and ...,2010.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,mortalityandpredictorsofdeath1monthand3yearsaf...,WOS:000273552100004,0.554120
99,678134,Weather factors are widely studied for their e...,Statistical Modeling Reveals the Effect of Abs...,2014.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,statisticalmodelingrevealstheeffectofabsoluteh...,WOS:000337735100017,0.957414


In [6]:
dais = pd.DataFrame.from_dict(DocAuthInst.objects.filter(doc__id__in=df['id'],institution__iregex='\w').values())
dais.head()

,id,doc_id,surname,initials,AU,AF,institution,position
0,1217824,562,None,None,"Pinkerton, KE","Pinkerton, Kent E.","Univ Calif Davis, Ctr Hlth & Environm, Davis, ...",4
1,1217742,562,None,None,"Thurston, GD","Thurston, George D.","NYU, Dept Environm Med, Sch Med, New York, NY ...",2
2,1217830,562,None,None,"Pinkerton, KE","Pinkerton, Kent E.","Univ Calif Davis, John Muir Inst Environm, Dav...",4
3,1217730,562,None,None,"Rice, MB","Rice, Mary B.","Harvard Univ, Beth Israel Deaconess Med Ctr, S...",1
4,1217695,562,None,None,"Rice, MB","Rice, Mary B.","Massachusetts Gen Hosp, Pulm & Crit Care Unit,...",1


'SC'

In [106]:
import pycountry

def clean_country(x):
    country = x.split(',')[-1]
    if "USA" in country:
        country = "USA"
    return country.strip('. ]"\'')
        
dais['country'] = dais['institution'].apply(lambda x: clean_country(x))
dais.loc[dais['country']=='CA 95616','country'] = "USA"
dais.loc[dais['country']=="Peoples R China",'country'] = "China"
dais.loc[dais['country']=="Rep of Georgia",'country'] = "Georgia"
dais.loc[dais['country']=="South Korea",'country'] = "Republic of Korea"
dais.loc[dais['country']=="Trinidad Tobago",'country'] = "Trinidad and Tobago"
dais.loc[dais['country']=="Trinid & Tobago",'country'] = "Trinidad and Tobago"
dais.loc[dais['country']=="North Ireland","country"] = "Northern Ireland"  
dais.loc[dais['country']=="DEM REP CONGO", "country"] = "Congo, The Democratic Republic of the"
dais.loc[dais['country']=="Mongol Peo Rep", "country"] = "Mongolia"
dais.loc[dais['country']=="Laos", "country"] = "Lao"
dais.loc[dais['country']=="Bosnia & Herceg", "country"] = "Bosnia"
dais.loc[dais['country']=="U Arab Emirates", "country"] = "United Arab Emirates"
dais.loc[dais['country']=="Cote Ivoire", "country"] = "Côte d'Ivoire"
dais.loc[dais['country']=="Fr Polynesia", "country"] = "French Polynesia"
dais.loc[dais['country']=="Papua N Guinea", "country"] = "Papua New Guinea"
dais.loc[dais['country']=="Rep Congo", "country"] = "Republic of the Congo"
dais.loc[dais['country']=="Fr Polynesia", "country"] = "French Polynesia"
dais.loc[dais['country']=="Swaziland", "country"] = "Eswatini"
dais.loc[dais['country']=="Neth Antilles", "country"] = "Netherlands Antilles"
dais.loc[dais['country']=="Guinea Bissau", "country"] = "Guinea-Bissau"
dais.loc[dais['country']=="Cape Verde", "country"] = "Cabo Verde"
dais.loc[dais['country']=="Cent Afr Republ", "country"] = "Central African Republic"
dais.loc[dais['country']=="Equat Guinea", "country"] = "Equatorial Guinea"
dais.loc[dais['country']=="Cent Afr Republ", "country"] = "Central African Republic"
dais.loc[dais['country']=="Cent Afr Republ", "country"] = "Central African Republic"
dais.loc[dais['country']=="Cent Afr Republ", "country"] = "Central African Republic"



dais['country'] = dais['country'].str.replace('St','Saint')
dais['country'] = dais['country'].str.replace('&','and')

for c in dais.country.unique():
    try:
        country = pycountry.countries.search_fuzzy(c)[0]
    except:
        country = pycountry.historic_countries.get(name=c)
        if country is None:
            print("Could not find it!") 
            break
    dais.loc[dais['country']==c,["alpha_3","name","alpha_2"]] = [country.alpha_3, country.name, country.alpha_2]
#list(dais.country.unique())

In [108]:
dais.to_csv('data/institutional_locations.csv',index=False)

In [36]:
dais[dais['country']=='CA 95616'].institution.unique()

array(["['UNIV CALIF DAVIS,DEPT MED & EPIDEMIOL,SCH VET MED,DAVIS,CA 95616.']"],
      dtype=object)